# Parsing Model Output with LangChain Prompt Templates

LangChain prompt templates not only help structure input prompts but also facilitate the parsing of model outputs. By defining a predictable format for the output within the prompt, developers can more easily extract specific information or structure responses in a way that is machine-readable. This capability is especially useful in applications that require post-processing or further analysis of the language model's response.











# Setting Up Work Environment

In [1]:
!pip install --upgrade google-generativeai

In [2]:
!pip install -q -U google-genai

In [3]:
!pip install langchain-community

In [4]:
!pip install -U langchain-google-genai

  Using cached google_ai_generativelanguage-0.6.18-py3-none-any.whl.metadata (9.8 kB)
Using cached google_ai_generativelanguage-0.6.18-py3-none-any.whl (1.4 MB)
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.15
    Uninstalling google-ai-generativelanguage-0.6.15:
      Successfully uninstalled google-ai-generativelanguage-0.6.15
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.18 which is incompatible.


In [5]:
import os
from google import genai
import google.generativeai as ggenai
from google.colab import userdata
from IPython.display import display
from IPython.display import Markdown

from PIL import Image
from google.genai import types

from IPython.display import HTML

In [23]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate

from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

In [7]:
# Set up the API key (Replace 'YOUR_API_KEY' with your actual Gemini API key)
key = userdata.get('genai_api')
client = genai.Client(api_key=key)

List the set of available models

In [8]:
ggenai.configure(api_key=key)

models = ggenai.list_models()
for model in models:
    print(model.name)

models/embedding-gecko-001
models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-002
models/gemini-1.5-pro
models/gemini-1.5-flash-latest
models/gemini-1.5-flash
models/gemini-1.5-flash-002
models/gemini-1.5-flash-8b
models/gemini-1.5-flash-8b-001
models/gemini-1.5-flash-8b-latest
models/gemini-2.5-pro-exp-03-25
models/gemini-2.5-pro-preview-03-25
models/gemini-2.5-flash-preview-04-17
models/gemini-2.5-flash-preview-05-20
models/gemini-2.5-flash
models/gemini-2.5-flash-preview-04-17-thinking
models/gemini-2.5-flash-lite-preview-06-17
models/gemini-2.5-pro-preview-05-06
models/gemini-2.5-pro-preview-06-05
models/gemini-2.5-pro
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-lite-preview-02-05
models/gemini-2.0-flash-lite-preview
models/gemini-2.0-pro-exp
models/gemini-2.0-pro-exp-02-05
models/gemini-exp-1206
model

Create a helper function to make it easier to use prompts and look at generated outputs.

In [9]:
def get_completion(prompt, model="gemini-1.5-flash", temperature = 0):
  response = client.models.generate_content(
      model=model,
      contents=prompt,
      config=types.GenerateContentConfig(max_output_tokens=200, temperature = temperature)
      )
  return response.text  # Extract the generated text

In [10]:
chat = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",       # or another Gemini model of your choice
    temperature=0.0,
    google_api_key=key# exact same temperature as your ChatOpenAI example
)

In [11]:
chat

ChatGoogleGenerativeAI(model='models/gemini-1.5-flash', google_api_key=SecretStr('**********'), temperature=0.0, client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x7bc30568e650>, default_metadata=(), model_kwargs={})

# Parsing Model Output with LangChain Prompt Templates

To clarify the concept of output parsing, let us begin with a practical example. In this example, we demonstrate how to instruct a language model (LLM) to generate output in JSON format, and then use LangChain to parse and handle that structured output effectively.

In [12]:
x = {
  "gift": False,
  "delivery_days": 5,
  "price_value": "pretty affordable!"
}

In [13]:
x.get("price_value")

'pretty affordable!'

In [14]:
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""

In [16]:
prompt_template = ChatPromptTemplate.from_template(review_template)

prompt_template

ChatPromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='For the following text, extract the following information:\n\ngift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.\n\ndelivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.\n\nprice_value: Extract any sentences about the value or price,and output them as a comma separated Python list.\n\nFormat the output as JSON with the following keys:\ngift\ndelivery_days\nprice_value\n\ntext: {text}\n'), additional_kwargs={})])

In [19]:
messages = prompt_template.format_messages(text=customer_review)

In [21]:
response = chat.invoke(messages)

print(response.content)

```json
{
  "gift": true,
  "delivery_days": 2,
  "price_value": ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]
}
```


- It is important to note that, although the response appears to be in JSON format with key-value pairs, its data type is actually a string. This means that the output is not a Python dictionary, but rather a single, continuous string that resembles **JSON**.

In [22]:
type(response.content)

str

- The objective is to access the `gift` key from the response content and retrieve its corresponding value, which is expected to be `true`. However, attempting to do this directly will result in an error, as the response is a string rather than a Python dictionary. Consequently, the string must first be parsed into a proper dictionary before key-based access is possible.

# Parsing the LLM Output String into a Python Dictionary

- Since the output generated by the language model is returned as a string, it must be converted into a Python dictionary in order to access its key-value pairs. This can be achieved using Python’s built-in `json` module, specifically the `json.loads()` function, which parses a valid JSON-formatted string into a dictionary object. This step is essential for structured data manipulation and further processing within the application.

- To enable structured parsing of the model's output, we define a set of response schemas that explicitly describe the expected fields. These include the gift schema, the delivery days schema, and the price value schema. Each schema includes a description that outlines the type and purpose of the corresponding data field, guiding the model to return output in a predictable and parsable format.

In [28]:
gift_schema = ResponseSchema(
    name="gift",
    description="Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown."
    )

delivery_days_schema = ResponseSchema(
    name="delivery_days",
    description="How many days did it take for the product to arrive? If this information is not found, output -1."
    )

price_value_schema = ResponseSchema(
    name="price_value",
    description="Extract any sentences about the value or price, and output them as a comma separated Python list."
    )

response_schemas = [gift_schema, delivery_days_schema, price_value_schema]

- After defining the schemas, LangChain can automatically generate the appropriate prompt instructions. This is achieved through the output parser, which provides guidance on how the language model should structure its response to align with the specified schemas. By printing the `format_instructions`, we can view the exact instructions that should be included in the prompt sent to the LLM.

In [29]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"gift": string  // Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.
	"delivery_days": string  // How many days did it take for the product to arrive? If this information is not found, output -1.
	"price_value": string  // Extract any sentences about the value or price, and output them as a comma separated Python list.
}
```


- This set of instructions is specifically designed to guide the language model in generating output that can be successfully parsed by the LangChain output parser. To implement this, we define a new review template, which incorporates the formatting instructions automatically generated by LangChain.

- Using this review template, we can construct a prompt and then create the corresponding message objects to be sent to the OpenAI endpoint. If needed, the complete prompt text can be examined—it contains clear directives instructing the model to extract the fields: `gift`, `delivery_days`, and `price_value`. Following the prompt, the formatting instructions ensure that the model’s response adheres to the expected structure, enabling reliable parsing.

In [30]:
review_template_2 = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product\
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

text: {text}

{format_instructions}
"""

In [31]:
prompt_temp_2 = ChatPromptTemplate.from_template(review_template_2)

In [33]:
messages = prompt_temp_2.format_messages(text=customer_review, format_instructions=format_instructions)
messages

[HumanMessage(content='For the following text, extract the following information:\n\ngift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.\n\ndelivery_days: How many days did it take for the productto arrive? If this information is not found, output -1.\n\nprice_value: Extract any sentences about the value or price,and output them as a comma separated Python list.\n\ntext: This leaf blower is pretty amazing.  It has four settings:candle blower, gentle breeze, windy city, and tornado. It arrived in two days, just in time for my wife\'s anniversary present. I think my wife liked it so much she was speechless. So far I\'ve been the only one using it, and I\'ve been using it every other morning to clear the leaves on our lawn. It\'s slightly more expensive than the other leaf blowers out there, but I think it\'s worth it for the extra features.\n\n\nThe output should be a markdown code snippet formatted in the following schema, including the 

- It is important to note that the parsed output is now of type dictionary, rather than a string. This allows us to directly access the values associated with specific keys for example, retrieving `true` from the `gift` key, `2` from the `delivery_days` key, and the corresponding value from the `price_value` key. This approach provides a convenient and structured way to convert LLM output into a Python dictionary, thereby making the data easier to access and manipulate for downstream processing tasks.

In [34]:
response = chat.invoke(messages)

In [37]:
type(response.content)

str

In [38]:
output_dict = output_parser.parse(response.content)

In [39]:
output_dict

{'gift': 'True',
 'delivery_days': '2',
 'price_value': "It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."}

In [40]:
type(output_dict)

dict

In [41]:
output_dict.get('delivery_days')

'2'